In [25]:
from sherpa.astro.ui import *
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [37]:
clean()
base='/Users/kciurleo/Downloads/0782940101PPS/pps/'

load_pha(f'{base}P0782940101PNS003SRSPEC0001.FTZ')


read ARF file /Users/kciurleo/Downloads/0782940101PPS/pps/P0782940101PNS003SRCARF0001.FTZ
read RMF file /Users/kciurleo/Downloads/0782940101PPS/pps/epn_e3_ff20_sdY9.rmf
read background file /Users/kciurleo/Downloads/0782940101PPS/pps/P0782940101PNS003BGSPEC0001.FTZ


In [24]:
os.chdir(base)
files = os.listdir()
print(len(files))
interesting_files=[]
for f in files:
    if "P0782940101PNS003" in f and 'FTZ' in f:
        interesting_files.append(f)
        print(f)
print(len(interesting_files))



863
P0782940101PNS003EXPMAP2000.FTZ
P0782940101PNS003IMAGE_3000.FTZ
P0782940101PNS003IMAGE_1000.FTZ
P0782940101PNS003SRCTSR802E.FTZ
P0782940101PNS003SRCTSR803C.FTZ
P0782940101PNS003SRCTSR8009.FTZ
P0782940101PNS003PIEVLI0000.FTZ
P0782940101PNS003SRCTSR8008.FTZ
P0782940101PNS003SRSPEC002E.FTZ
P0782940101PNS003IMAGE_5000.FTZ
P0782940101PNS003BGSPEC000D.FTZ
P0782940101PNS003BGSPEC0002.FTZ
P0782940101PNS003BGSPEC0016.FTZ
P0782940101PNS003BGSPEC0017.FTZ
P0782940101PNS003BGSPEC0003.FTZ
P0782940101PNS003BGSPEC000E.FTZ
P0782940101PNS003EXPMAP8000.FTZ
P0782940101PNS003SRSPEC0008.FTZ
P0782940101PNS003BGSPEC001C.FTZ
P0782940101PNS003BGSPEC0015.FTZ
P0782940101PNS003BGSPEC0001.FTZ
P0782940101PNS003EXPMAP4000.FTZ
P0782940101PNS003BGSPEC0014.FTZ
P0782940101PNS003BGSPEC000F.FTZ
P0782940101PNS003BGSPEC0028.FTZ
P0782940101PNS003SRSPEC0009.FTZ
P0782940101PNS003SRSPEC003C.FTZ
P0782940101PNS003SRCARF003C.FTZ
P0782940101PNS003SRCARF0009.FTZ
P0782940101PNS003BGSPEC0010.FTZ
P0782940101PNS003BGSPEC0004.FTZ
P078

In [36]:
sas_vers = fits.getheader(f'{base}P0782940101PNS003SRSPEC0001.FTZ',ext=0)['SASVERS']
sas_vers

'xmmsas_20190401_1820-18.0.0'